In [1]:
import numpy as np
import pandas as pd


from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

from psql_functions import execQuery
from miss_data import add_missing_dates, add_missing_counts
from make_plots import dict_error_plot_epsilons, dict_error_plot_n

import warnings
warnings.filterwarnings('ignore')

import sys
import os
import re
owd = os.getcwd()

In [2]:
param_dic = {
    "host"      : "localhost",
    "database"  : "bachelorBesoeg2014",
    "user"      : "postgres",
    "password"  : "password",
    "port"      : "5432"
}

query = """select time_ from _775147;"""
result = execQuery(param_dic, query)
dates = [(date[0]) for date in result]

query = """select count_ from _775147;"""
result = execQuery(param_dic, query)

counts = [(count[0]) for count in result]

all_dates = add_missing_dates(dates)
all_counts =  add_missing_counts(counts, dates, all_dates)

n_data_structures = 50


Executed query and closed connection.
Executed query and closed connection.


In [3]:
onlydirs = [f for f in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(os.getcwd(), f)) and f[0] != '.']

In [4]:
onlydirs

['Data', 'plots', 'range_queries', 'results', '__pycache__']

In [5]:
def make_lookup_error_dict_e_n(epsilons, n):
    epsilon_dict = dict.fromkeys(epsilons)
    n_dicts = []
    for i in range(0,len(epsilons)):
        n_dict = dict.fromkeys(n)
        n_dicts.append(n_dict)

    i = 0
    for e_key in epsilon_dict:
        epsilon_dict[e_key] = n_dicts[i]
        i = i + 1
    return epsilon_dict

def make_query_lookup_dict_n_r(n, rs):
    n_dict = dict.fromkeys(n)
    r_dicts = []
    for i in range(0,len(n)):
        #print(rs[i])
        r_dict = dict.fromkeys(rs[i])
        r_dicts.append(r_dict)

    for idx, n_key in enumerate(n_dict):
            n_dict[n_key] = r_dicts[idx]
    
    return n_dict

def fill_up_query_lookup_dict_n_r(n, rs, files):
    dict_ = make_query_lookup_dict_n_r(n, rs)
    for idx, f in enumerate(files):
        here_n = re.search(r'N=\d+', f).group(0)
        here_n = here_n.split("N=")[1]
        here_r = re.search(r'r=\d+', f).group(0) 
        here_r = here_r.split("r=")[1]
        
        querries = load_range_queries_n_split(f, n_data_structures)
        dict_[int(here_n)][int(here_r)] = querries.copy()
    return dict_

In [6]:
def make_lookup_error_dict(epsilons, n, rs):
    epsilon_dict = dict.fromkeys(epsilons)
    n_dicts = []
    for i in range(0,len(epsilons)):
        n_dict = dict.fromkeys(n)
        n_dicts.append(n_dict)
    r_dicts = []
    for i in range(0,len(n)*len(epsilons)):
        r_dict = dict.fromkeys(rs[i%len(n)])
        r_dicts.append(r_dict)
    i = 0
    j = 0
    for e_key in epsilon_dict:
        epsilon_dict[e_key] = n_dicts[i]
        n_dict = epsilon_dict[e_key]
        for n_key in n_dict:
            #print(j)
            n_dict[n_key] = r_dicts[j]
            j = j + 1
        i = i + 1
    return epsilon_dict


def fill_up_lookup_error_dict(epsilons, n, degrees, files):
    dict_ = make_lookup_error_dict(epsilons, n, degrees)
    for idx, f in enumerate(files):
        e = re.search(r'e=[-+]?[0-9]*\.?[0-9]*', f).group(0)
        e = e.split("e=")[1]
        here_n = re.search(r'N=\d+', f).group(0)
        here_n = here_n.split("N=")[1]
        here_r = re.search(r'r=\d+', f).group(0) 
        here_r = here_r.split("r=")[1]

        dict_[float(e)][int(here_n)][int(here_r)] = np.loadtxt(f, delimiter=',')
    return dict_

def find_error_different_queries(est, cor):
    erorrs = est - cor
    max_error = np.amax(erorrs)
    min_error = np.amin(erorrs)
    rmse = mean_squared_error(cor, est, squared = False)
    error_dict = {'rmse': rmse, 'max': max_error, 'min': min_error, 'abs': np.mean(erorrs)}
    return error_dict

In [8]:
len(epsilons)*len(n)*len(rs[0])

336

In [15]:
epsilons = np.array([1.2, 1, 0.8])
n = np.array([32, 256, 512])

# Loading the data 

In [9]:
#C:\Users\Jonas Friis\Documents\Universitet\Bachelors-Thesis\src\results\sample_querys\local_hh
os.chdir(os.getcwd()+'/'+'results/full_range/')
os.getcwd()

'C:\\Users\\Jonas Friis\\Documents\\Universitet\\Bachelors-Thesis\\src\\results\\full_range'

In [18]:
files = [f for f in os.listdir('.')]
files

['cor_e=1.2_N=32.csv', 'flat_est_e=1.2_N=32.csv', 'hh_est_e=1.2_N=32.csv']

In [19]:
make_lookup_error_dict_e_n(epsilons, n)

{1.2: {32: None, 256: None, 512: None},
 1.0: {32: None, 256: None, 512: None},
 0.8: {32: None, 256: None, 512: None}}

In [29]:
cor = np.loadtxt(files[0], delimiter=',')
cor

array([6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081.])

In [30]:
flat = np.loadtxt(files[1], delimiter=',')
flat

array([6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081., 6081.,
       6081., 6081., 6081., 6081., 6081.])

In [31]:
hh = np.loadtxt(files[2], delimiter=',')
hh

array([6426., 5730., 6054., 6180., 6174., 5988., 5868., 6408., 6234.,
       6168., 6120., 6048., 5790., 6360., 6174., 5910., 6264., 5742.,
       6390., 6408., 6390., 5820., 6012., 6150., 5910., 5916., 5994.,
       6084., 5970., 6072., 6126., 5922., 6276., 6018., 6258., 6252.,
       5988., 6228., 5844., 6144., 6096., 5850., 5832., 6198., 6048.,
       5934., 5970., 6048., 6240., 6258.])

In [ ]:
error_dict = make_lookup_error_dict(epsilons, n, rs)

for e in epsilons:
    for N in n:
        #r_values = error_dict[e][N]
        for r in error_dict[e][N]:
            error_dict[e][N][r] = find_error_different_queries(est_dict[e][N][r], cor_dict[e][N][r])

In [ ]:
def get_answers_for_r(n_r, step_size, all_answers):
    """
    Setup of the datastructere
    Parameters:
    n_r (int): Which index/number of the r values
    step_size (int): How many r values there is 
    all_answers (Array): All answers for all the differnt r values
    Returns:
    The answers for this specific r
    """
    return all_answers[n_r::step_size]

In [ ]:
dict_plot_rmse_dict = make_lookup_error_dict_e_n(epsilons, n)
for e in epsilons:
    for N in n:
        rmse = []
        for r_keys in error_dict[e][N]: 
            rmse.append(error_dict[e][N][r_keys]['rmse'])
        
        dict_plot_rmse_dict[e][N] = rmse

In [ ]:
dict_plot_rmse_dict

In [ ]:
def plot(xs, ys, N, epsilon, plot_name):

    plt.scatter(xs, ys)
    #plt.hlines(np.mean(answears128),line_range[0],line_range[1])
    #plt.hlines(np.mean(correct_answear),line_range[0],line_range[1])
    #plt.legend()
    plt.xlabel('Length of query')
    plt.ylabel('RMSE')
    plt.title(f'Flat OLH with N = {N} and \u03B5 = {epsilon}')
    #plt.savefig(plot_name,bbox_inches='tight')
    plt.show()

In [ ]:
for e in epsilons:
    for N in n:
        xs = list(error_dict[e][N].keys())
        print(xs)
        ys = dict_plot_rmse_dict[e][N]
        print(ys)
        plot(xs, ys, N, e, 'e')


In [ ]:
def same_parameter(answers, correct_answer):
    error = np.abs(correct_answer-answers)
    max_error = np.amax(error)
    min_error = np.amin(error)
    true_values = np.full(len(answers), correct_answer)
    mse = mean_squared_error(true_values, answers, squared = False)
    error_dict = {'mse': mse, 'max': max_error, 'min': min_error, 'abs': np.mean(error)}
    
    return error_dict


In [ ]:
print(flat_empty)
print(len(flat_empty))
print(hh_empty)
print(len(hh_empty))

In [ ]:
epsilon_error_dicts = {}
for idx, e in enumerate(epsilons):
    epsilon_error_dicts[e] = same_parameter(epsilons_res[idx], correct_answer)

In [ ]:
epsilon_error_dicts

In [ ]:
plt_name = 'flat_freq_epsilon'
dict_error_plot_epsilons(epsilon_error_dicts,plt_name, epsilons)

In [ ]:
n_error_dicts = {}
for idx, N in enumerate(n):
    n_error_dicts[N] = same_parameter(ns[idx], correct_answer)



In [ ]:
n_error_dicts

In [ ]:
print(n)
plt_name = 'flat_freq_N'
dict_error_plot_n(n_error_dicts,plt_name, n)